In [19]:
# pip install pymorphy2 >> None

In [1]:
import pandas as pd
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# задаем директорию
file_path = "C:/Users/user/Desktop/Хакатоны/LinkedIn menthoring"

In [3]:
# открываем файлы с данными
#posts = pd.read_csv(f'{file_path}/posts.csv')
#profiles = pd.read_csv(f'{file_path}/profiles.csv')
posts = pd.read_csv('posts.csv')
profiles = pd.read_csv('profiles.csv')

In [4]:
# открываем датафрейм с постами
posts.head(2)

,Unnamed: 0,user_id,text,likes,comments,reposts
0,0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [5]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,Unnamed: 0,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [6]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [7]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [8]:
# удалим посты на украинском языке

# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]


# определяем шаблон для украинских символов
# ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# # сохраняем в датафрейме только те строки, в которых столбец "post" не содержит текста на украинском языке
# posts = posts[~posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)]

In [9]:
posts.head()

,Unnamed: 0,user_id,post,likes,comments,reposts
0,0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0
2,2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0
3,3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0
4,4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0


In [10]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [11]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', '', x))

In [12]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [13]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)

# только русские буквы и пробелы
# posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', '', regex=True) 

# только русские, английские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'[^а-яА-ЯёЁa-zA-Z\s]', '', x))


# Шаблон '\W' в методе str.replace() представляет любой символ, не состоящий из слов, в т.ч. пробелы.
# Шаблон '[^\w\s]' пробелы оставляет, '[^\w\s\d]' + цифры

In [15]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
current_directory = os.getcwd()
#file_path_words = f'{file_path}\stop_words.txt'
file_path_words = 'stop_words.txt'
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krasn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krasn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [17]:
'''если НЕ оставляем в постах английские слова'''
# # определяем шаблон регулярного выражения для русских букв
# pattern = '[^а-яА-ЯёЁ]'
# # создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
# mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# # фильтруем датафрейм, используя маску
# posts = posts[mask]

'''если оставляем в постах английские слова'''
pattern = '[^а-яА-ЯёЁ]'
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
posts = posts[mask]

english_letters_mask = posts['post_lemmatized'].str.match(r'^[a-zA-Z\s]+$')
posts = posts[~english_letters_mask]

In [18]:
posts.sample(20)

,Unnamed: 0,user_id,post,likes,comments,reposts,post_lemmatized
1126,1126,enjirouz,"Cколько на самом деле зарабатывают программисты на старте карьеры? А главное - как быстрее повысить свою зарплату на 20-30, а то и 40% за год? Смотри видео до конца, и узнаешьhttps://lnkd.in/eKtw2shКстати, для тех, кто только начинает свой путь в IT или хочет сменить направление - ловите курсы от онлайн-школы Skillbox со скидкой до 50%: https://lnkd.in/e9aYzVFА кто хочет попробовать прежде, чем покупать - ловите бесплатный доступ на 7 дней к онлайн-курсам и интенсивам Skillbox: https://lnkd.in/eAFG2B4#ityoutubersru #зарплата #программист",3,1,0,cколько дело зарабатывать программист старт карьеры главное быстрый повысить зарплата год смотреть видео конца узнаешьhttpslnkdinektwshкстати начинать путь хотеть сменить направление ловить курс онлайн школа skillbox скидка httpslnkdineayzvf хотеть попробовать прежде покупать ловить бесплатный доступ день онлайн курс интенсив skillbox httpslnkdineafgb
2391,2391,nikolay-schwartz,"!!! Очень удобная шпаргалка по GIT !!! Тех кто еще не сделал это, приглашаю подписаться на мой телеграмм-канал QALife https://t.me/QA4Life.Это канал для начинающих и опытных тестировщиков, QA и QC, а также будет полезен также для PO, PM, DA, BA и SM специалистов . Здесь я ежедневно делюсь ещё большим количеством материалов по IT тематике (статьи, обучающие видео, новости, ИТ-юмор, опросы, обсуждения…)#IT #engineer #software #qa #qc #quality #recruiter #recruiting #HR #tester #qa_engineer #testing #PM #BA #DA #тестирование #development #developer #тестировщик #тестер",6,0,0,очень удобный шпаргалка git ещё сделать приглашать подписаться телеграмм канал qalife httpstmeqalifeэтый канал начинающий опытный тестировщиков qa qc также полезный также po pm da ba sm специалист ежедневно делиться ещё больший количество материал тематика статьи обучать видео новости юмор опросы обсуждения
1326,1326,andrey-mashukov,https://lnkd.in/dqgMKkPМой личный проектсайт: http://farento.com,0,0,0,httpslnkdindqgmkkpма личный проектсайт httpfarentocom
1378,1378,%D0%B0%D0%BD%D0%B3%D0%B5%D0%BB%D0%B8%D0%BD%D0%B0-%D0%BE%D0%BD%D0%B4%D0%B0%D1%80-834b41227,Мой первый полноценный проект Сайт для салона красоты:) #вебдизайнер #дизайнер #графическийдизайнер #графическийдизайн #dribbble #behance #вебдизайн #uidesigner #langingpage #фриланс #дизайнсайтов #дизайн #design #ux #ui #webdesign #webdesigner #дизайнсайта #figma #opentowork,0,0,0,первый полноценный проект сайт салон красоты
3453,3453,%D0%B5%D1%80%D0%BB%D0%B0%D0%BD-%D0%B0%D1%85%D0%BC%D0%B5%D1%82%D0%BE%D0%B2-32532258,"Кто такой питчер?Это человек, который подает мяч в бейсболе. От этого слова произошло другое - питч, по иному - краткая презентация своего продукта. Питч должен быть такой же быстрый как мяч, который бросает питчер и также как мяч попасть в цель. Для этого нужно проделать большую домашнюю работу и не полагаться на импровизацию.Есть полезное упражнение, которое позволит донести суть вашего проекта за короткое время. Тут работает закон Парето - если взять обычную презентацию на 15 минут, то суть (80% важной информации) можно рассказать за 20% времени (3 минуты). Я проводил очень много презентаций по разным продуктам и проектам. И чем выше уровень слушателя, речь идет о масштабах компании и должности человека в нем, тем меньше“воды” и выше скорость он просил. Тогда я на основе обычной презентации, останавливаясь на нужных слайдах, за 2-3 минуты описывал проект. Но как к такому подготовиться? Ниже я описал три простых пункта:1⃣Сначала определите направление вашего проекта. Вы должны сказать одно вводное предложение, которое задаст направление всего дальнейшего выступления. Чтобы услышав его, человек понял, в каком направлении его бизнеса это может быть применимо. Бывали ситуации, когда после этого предложения руководитель останавливал меня и звал своих ответственных в кабинет.2⃣Определите, как устроен ваш проект. Обычно это занимает 2 слайда. Если речь идет о готовом продукте, первый слайд описывает пр

In [19]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2168 entries, 0 to 8597
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2168 non-null   int64 
 1   user_id          2168 non-null   object
 2   post             2168 non-null   object
 3   likes            2168 non-null   object
 4   comments         2168 non-null   int64 
 5   reposts          2168 non-null   int64 
 6   post_lemmatized  2168 non-null   object
dtypes: int64(3), object(4)
memory usage: 135.5+ KB


Объединяем датафреймы

In [20]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [21]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [22]:
# удаляем лишние столбцы
df = df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [23]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [24]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2156 entries, 0 to 2167
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2156 non-null   object
 1   post              2156 non-null   object
 2   likes             2156 non-null   int64 
 3   comments          2156 non-null   int64 
 4   reposts           2156 non-null   int64 
 5   post_lemmatized   2156 non-null   object
 6   user_name         2156 non-null   object
 7   user_head         2156 non-null   object
 8   user_work         2064 non-null   object
 9   user_position     2156 non-null   object
 10  user_tags         412 non-null    object
 11  user_location     2149 non-null   object
 12  user_viewers      2156 non-null   object
 13  user_contacts     2156 non-null   object
 14  user_common_info  1961 non-null   object
dtypes: int64(3), object(12)
memory usage: 269.5+ KB


In [26]:
df.sample(5)

,user_id,post,likes,comments,reposts,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
1391,votumam,"Если меня спросят, что мне больше всего нравится в моей работе, отвечу - исследование пользователей.Если посчитать суммарно, то я провела около 250 глубинных интервью (custdev) и именно общение с юзерами приносит мне самое большее удовольствие.Самое странное интервью у меня было в челюстно-лицевой больнице, там работал наш клиент. А самое смешное в кинотеатре, оно прошло без пользователя Оказалось, что когда ребята назначали время для интервью, они не уточнили о том, из какого города пользователь. И как оказалось, в Алматы и Астане есть 2 абсолютно одинаковых ТРЦ, с одинаковыми кинотеатрами внутри. Представляете наше с клиентом удивление, когда спустя 15 минут попыток найти друг друга по телефону, мы все-таки выяснили, что находимся в разных городах. Почти как в ""Иронии судьбы""Хочу немножко подробнее рассказать о глубинных исследованиях. Наверное нет смысла рассказывать зачем их проводить, расскажу как я их провожу.С кем разговаривать?Для начала сформулируйте проблему. Ну например, вы столкнулись с проблемой удержания пользователей и бОльшая часть клиентов совершает свой первый заказ, но не возвращается снова. Вытащите всех клиентов, у которых был всего 1 заказ, за какое-то определенное время, позвоните и договоритесь на встречу.Где разговаривать?Раньше я общалась оффлайн, сейчас стараюсь это делать онлайн, по миту или зуму. Так получается гораздо эффективнее, как минимум, экономишь время на дорогу. Но обязательно делать это с видео, не нужно просто звонить клиенту. Вам важно видеть эмоции пользователя.Как долго разговаривать и сколько вопросов задавать?Обычно я пищу себе черновик с 8-10 вопросами, но в процессе разговора, стараюсь ""раскрыть пользователя"" и задаю гораздо больше вопросов. Интервью обычно длится около 30-40 минут.Нужно ли платить за интервью?Да, в психологии есть очень хороший принцип ""брать-давать"". Вы ""забираете"" у клиента его время и должны дать что-то взамен. Это могут быть деньги, бонусы, какие-то плюшки, в общем вы точно должны компенсировать клиенту его время.Очень долго могу говорить о теме глубинных исследований. Пишите, если хочется узнать что-то конкретное, а я в следующий раз расскажу о том, какие вопросы нужно и какие не нужно задавать в процессе интервью.#it #product #custdev",30,6,1,спросят большой нравиться работе ответить исследование пользователейести посчитать суммарно провести около глубинный интервью custdev именно общение юзер приносить самый больший удовольствиесамый странный интервью челюстно лицевой больнице работать клиент самый смешной кинотеатре пройти пользователь оказалось ребята назначать время интервью уточнить город пользователь оказалось алматы астана абсолютно одинаковый трц одинаковый кинотеатр внутри представлять клиент удивление спустя минута попытка найти друг друг телефону всё выяснили находиться разный городах ирония судьбыхотать немножко подробный рассказать глубинный исследованиях наверное смысл рассказывать проводить рассказать провожус разговариватьдлить начало сформулировать проблему например столкнуться проблема удержание пользователь больший часть клиент совершать первый заказ возвращаться снова вытащить клиентов заказ определённый время позвонить договориться встречугда разговариватьраньша общаться оффлайн стараться делать онлайн мит зуму получаться гораздо эффективнее минимум экономить время дорогу обязательно делать видео нужно просто звонить клиенту важно видеть эмоция пользователякак долго разговаривать сколько вопрос задаватьобычно пища черновик вопросами процесс разговора стараться раскрыть пользователя задавать гораздо большой вопросов интервью обычно длиться около минутнужный платить интервьюда психология очень хороший принцип брать давать забираете клиент время должный дать взамен деньги бонусы плюшки общий точно должный компенсировать клиент времяочень долго говорить тем